In [1]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql.window import Window
import time
import shutil
from pyspark.sql.functions import current_timestamp, col, unix_timestamp

from datetime import datetime, timedelta


import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


24/12/11 23:57:45 WARN Utils: Your hostname, MacBook-Pro-cua-My.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/12/11 23:57:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ibulmnie/.ivy2/cache
The jars for the packages stored in: /Users/ibulmnie/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c0a48bcd-762d-46e4-aa53-62f5863b6a20;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:

In [23]:
df = spark.read.parquet("gs://indicator-crypto/sma_results/BTC_CLOSE/part-00000-454539e1-d2c4-4e53-b14f-49d85d8b2d53-c000.snappy.parquet")

In [24]:
df.show(truncate=False)

+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+----------+---------+----------+---------+---------+-----------+----------+---------+--------------+---------------+---------------+---------------+----------------+----------------+
|BASE|DATE|OPEN|HIGH|LOW|CLOSE|VOLUME|YEAR|MONTH|__index_level_0__|BTC_CLOSE|ETH_CLOSE|USDT_CLOSE|BNB_CLOSE|USDC_CLOSE|XRP_CLOSE|ADA_CLOSE|MATIC_CLOSE|DOGE_CLOSE|SOL_CLOSE|SMA5_BTC_CLOSE|SMA10_BTC_CLOSE|SMA20_BTC_CLOSE|SMA50_BTC_CLOSE|SMA100_BTC_CLOSE|SMA200_BTC_CLOSE|
+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+----------+---------+----------+---------+---------+-----------+----------+---------+--------------+---------------+---------------+---------------+----------------+----------------+
+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+----------+---------+----------+---------+---------+-----------+----------+---------+--------------+---

In [20]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql import functions as F

# Define schema
schema = StructType([
    StructField("BASE", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("OPEN", FloatType(), True),
    StructField("HIGH", FloatType(), True),
    StructField("LOW", FloatType(), True),
    StructField("CLOSE", FloatType(), True),
    StructField("VOLUME", FloatType(), True),
    StructField("YEAR", StringType(), True),
    StructField("MONTH", StringType(), True),
    StructField("__index_level_0__", StringType(), True),
    StructField("BTC_CLOSE", FloatType(), True),
    StructField("ETH_CLOSE", FloatType(), True),
    StructField("USDT_CLOSE", FloatType(), True),
    StructField("BNB_CLOSE", FloatType(), True),
    StructField("USDC_CLOSE", FloatType(), True),
    StructField("XRP_CLOSE", FloatType(), True),
    StructField("ADA_CLOSE", FloatType(), True),
    StructField("MATIC_CLOSE", FloatType(), True),
    StructField("DOGE_CLOSE", FloatType(), True),
    StructField("SOL_CLOSE", FloatType(), True),
    StructField("SMA5_BTC_CLOSE", FloatType(), True),
    StructField("SMA10_BTC_CLOSE", FloatType(), True),
    StructField("SMA20_BTC_CLOSE", FloatType(), True),
    StructField("SMA50_BTC_CLOSE", FloatType(), True),
    StructField("SMA100_BTC_CLOSE", FloatType(), True),
    StructField("SMA200_BTC_CLOSE", FloatType(), True),
])

# Read stream with the defined schema
df = spark.readStream.schema(schema).parquet("gs://indicator-crypto/sma_results/BTC_CLOSE")

# Convert the 'DATE' column to timestamp if it's a string type
df = df.withColumn("DATE", F.to_timestamp("DATE", "yyyy-MM-dd"))

# Write the stream to a sink (e.g., console, file, etc.)
query = df.writeStream.outputMode("append").format("console").start()

# Wait for the streaming to finish
query.awaitTermination()


24/12/12 00:11:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/temporary-d2bb9187-d7e2-4370-9037-020d182d1f72. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/12 00:11:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+----------+---------+----------+---------+---------+-----------+----------+---------+--------------+---------------+---------------+---------------+----------------+----------------+
|BASE|DATE|OPEN|HIGH|LOW|CLOSE|VOLUME|YEAR|MONTH|__index_level_0__|BTC_CLOSE|ETH_CLOSE|USDT_CLOSE|BNB_CLOSE|USDC_CLOSE|XRP_CLOSE|ADA_CLOSE|MATIC_CLOSE|DOGE_CLOSE|SOL_CLOSE|SMA5_BTC_CLOSE|SMA10_BTC_CLOSE|SMA20_BTC_CLOSE|SMA50_BTC_CLOSE|SMA100_BTC_CLOSE|SMA200_BTC_CLOSE|
+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+----------+---------+----------+---------+---------+-----------+----------+---------+--------------+---------------+---------------+---------------+----------------+----------------+
+----+----+----+----+---+-----+------+----+-----+-----------------+---------+---------+------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 